In [ ]:
from __future__ import unicode_literals
#from gensim.summarization import keywords
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import pdb
import string
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
import unicodedata
!pip install sumy
import re

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

df =pd.read_csv('./JD_DataScience_USA_small_1.csv', encoding= 'unicode_escape')
df1 = pd.read_csv('./Resume_DataScience_Updated.csv', encoding= 'unicode_escape')


!pip install wget


import wget

wget.download("https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py",out="helper_prabowo_ml.py")

from helper_prabowo_ml import clean_html, remove_links, remove_special_characters, removeStopWords, remove_, remove_digits, lower, email_address, non_ascii, punct








[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 516.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21696 sha256=6d42c591a6f29f86ba38d6b9b58c1ea790828b6731e1e236a7ec7ff8f89d154d
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=c70a628bc699c26193999a750c44f96610a367f9db16fd55e904f693a77462c6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=106

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = df.dropna()
df1 = df1.dropna()

In [ ]:
def text_preprocess(data,col):
    data[col] = data[col].apply(func=clean_html)
    #data[col] = data[col].apply(func=remove_)
    #data[col] = data[col].apply(func=removeStopWords)
    #data[col] = data[col].apply(func=remove_digits)
    data[col] = data[col].apply(func=remove_links)
    data[col] = data[col].apply(func=remove_special_characters)
    data[col] = data[col].apply(func=punct)
    #data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=lower)
    return data


#print(resume[0])

In [ ]:
df = text_preprocess(df,'description')
df1 = text_preprocess(df1,'Resume')



<ipython-input-3-be09d5215647>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].apply(func=clean_html)
<ipython-input-3-be09d5215647>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].apply(func=remove_links)
<ipython-input-3-be09d5215647>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
jd = df['description'].tolist()
companies = df['company'].tolist()
positions = df['title'].tolist()

resume = df1['Resume'].tolist()

jd.append(resume[0])

In [ ]:
#models=[DecisionTreeRegressor(),LinearRegression(),RandomForestRegressor(),KNeighborsRegressor(n_neighbors = 5),BaggingRegressor(),LinearSVR()]
model_names=['bert-base-nli-mean-tokens','bert-base-nli-max-tokens','bert-base-nli-cls-token','bert-large-nli-mean-tokens','distiluse-base-multilingual-cased','distiluse-base-multilingual-cased-v2','xlm-r-bert-base-nli-stsb-mean-tokens','distilbert-base-nli-mean-tokens','roberta-base-nli-mean-tokens','roberta-large-nli-mean-tokens','roberta-base-nli-stsb-mean-tokens','roberta-large-nli-stsb-mean-tokens','distilbert-base-nli-mean-tokens','paraphrase-MiniLM-L6-v2','paraphrase-xlm-r-multilingual-v1','paraphrase-multilingual-MiniLM-L12-v2','paraphrase-multilingual-mpnet-base-v2','gtr-t5-large','all-MiniLM-L6-v2']

In [ ]:
### Easy — Sentence-Transformers
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

cosinesimilar = []
for model in model_names:
  model = SentenceTransformer(model)
  sentence_embeddings = model.encode(jd)
  sentence_embeddings.shape
  val = cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:])
  cosinesimilar.append(round(val[0, 0], 2))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2ec8bdcdfdf603d3293f385e5049f3b0fe32e7e3a121d328bb8aa53452fa2596
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
cosinesimilar

[0.77,
 0.86,
 0.72,
 0.77,
 0.41,
 0.41,
 0.66,
 0.72,
 0.8,
 0.58,
 0.48,
 0.59,
 0.72,
 0.45,
 0.46,
 0.48,
 0.4,
 0.79,
 0.41]

In [ ]:
import numpy as np


# Convert cosine similarity to cosine distance
distance_list = [1 - sim for sim in cosinesimilar]

# Print the results
print("Cosine Similarity:", cosinesimilar)
cosinedistance = [round(v, 2) for v in distance_list]
print("Cosine Distance:", cosinedistance )

Cosine Similarity: [0.77, 0.86, 0.72, 0.77, 0.41, 0.41, 0.66, 0.72, 0.8, 0.58, 0.48, 0.59, 0.72, 0.45, 0.46, 0.48, 0.4, 0.79, 0.41]
Cosine Distance: [0.23, 0.14, 0.28, 0.23, 0.59, 0.59, 0.34, 0.28, 0.2, 0.42, 0.52, 0.41, 0.28, 0.55, 0.54, 0.52, 0.6, 0.21, 0.59]
